关于Pagerank的原理，可以参考之前的实现[链接1>>](https://nbviewer.jupyter.org/github/zhulei227/ML_Notes/blob/master/notebooks/12_07_PGM_%E9%A9%AC%E5%B0%94%E7%A7%91%E5%A4%AB%E9%93%BE_PageRank%E7%AE%97%E6%B3%95.ipynb)  [链接2>>](https://nbviewer.jupyter.org/github/zhulei227/GNN_Notes/blob/master/04_DGL_02_%E5%AE%9E%E7%8E%B0PageRank.ipynb)  
下面随机构造一个图，并做实现

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@4666bab3
sc = org.apache.spark.SparkContext@664ef092


org.apache.spark.SparkContext@664ef092

In [2]:
//随机构造一个图
val initGraph=util.GraphGenerators.rmatGraph(sc,32,64) //随机初始化一个节点数最多为32，边数最多为64的图

initGraph = org.apache.spark.graphx.impl.GraphImpl@5d7f47de


org.apache.spark.graphx.impl.GraphImpl@5d7f47de

In [3]:
initGraph.vertices.count()

16

In [4]:
//pregel实现
val selfGraph=Pregel(initGraph.mapVertices((_,vd)=>(vd,1.0/16)),1.0/16,100)( //16是节点数
    vprog=(_,vd,aggMsg)=>(vd._1,0.85*aggMsg+0.15/16), //这里设置resetProb=0.15
    sendMsg=(edge)=>Iterator((edge.dstId,edge.srcAttr._2/edge.srcAttr._1)),
    mergeMsg=(a,b)=>a+b
).mapVertices((_,vd)=>vd._2*16)

selfGraph = org.apache.spark.graphx.impl.GraphImpl@7f066875


org.apache.spark.graphx.impl.GraphImpl@7f066875

In [5]:
//计算结果
selfGraph.vertices.take(10)

Array((16,2.022300157203121), (22,1.167746470235807), (28,1.023335068036767), (30,0.8420814883333414), (24,0.8431845098653021), (18,1.3325951806895024), (20,1.1517014028419028), (26,0.710455927099921), (19,2.1114639088127105), (21,1.0465220830607251))

In [6]:
//官方api结果
initGraph.pageRank(tol=0.001,resetProb=0.15).vertices.take(10)

Array((16,2.0217148260788047), (22,1.1665910003289264), (28,1.0235198444017368), (30,0.8423639700025821), (24,0.8429749210336648), (18,1.331420403407613), (20,1.1518802564211323), (26,0.7113296025490913), (19,2.1093915011745707), (21,1.046301681892569))